In [ ]:
! pip install langchain
! pip install transformers
! pip install accelerate

In [2]:
from huggingface_hub import login
login('')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


llm=HuggingFacePipeline(pipeline=pipeline,pipeline_kwargs={"max_new_tokens": 100},model_kwargs={'temperature':0.5})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
from langchain.prompts import PromptTemplate

NER_prompt = PromptTemplate(
    template='''You are a helpful assistant that performs Named Entity Recognition and the relations between the Organizations Names and the location if it has on text.
    Specifically,Organizations Names and locations.  please return  all Organizations Names, locations and the all relations between the entities  from this {text}''',
    input_variables=["text"]
)



NER_chain = NER_prompt | llm

In [6]:
text = '''Medical device manufacturer Millar Inc. has officially opened its new headquarters in Pearland,
six years after announcing its intention to move from its location in Houston’s East End. The new building is on a 5-acre tract at 11950 N. Spectrum Blvd.
in Pearland’s Lower Kirby District. Houston-based Clay Development and Construction delivered the 56,000-square-foot research and development, manufacturing and warehousing facility on Aug. 1, and Millar has since moved its finance,
marketing, business development and some of its human resources department into it, according to company President and CEO Tim Daugherty. The company develops and manufactures pressure sensor-enabled medical devices,
including catheters for life sciences and clinical applications. Millar hopes to move its production and engineering operations and the rest of its nearly 150 employees and contractors from its 29,730-square-foot building at 6001-A Gulf Freeway by the end of November.
The idea is to sell that building, which the Millar family has owned since 1988, according to the Harris County Appraisal District. “We were out of space for engineers and professional staff in the old building,
and then our cleanroom was basically getting close to full capacity,” Daugherty said. “And so with the different customers in our pipeline as well as product development that we're developing under the Millar brand, we needed more cleanroom space. So this was a big step to allow us to continue to grow.”
The new facility is designed to accommodate up to 220 employees, he said, a number the company anticipates reaching over the next two to three years. Millar first started working with the city of Pearland and the Pearland Economic Development Corp. to relocate its headquarters to accommodate Millar's growth in 2017.
The company purchased the property from the PEDC, which offered Millar up to $975,000 in cash incentives, in 2019 and broke ground on the facility in April 2022. Pearland made the most sense for a relocation, Daugherty said, because a lot of Millar’s current employees live in that area.
“When we looked for facility space in Houston, we did a heat map of all our employees,” he said. “And Pearland, from a commuter standpoint, was one of the closest areas for our highest concentration of employees.” Millar, the PEDC and the city of Pearland celebrated the grand opening of the new headquarters on Aug. 15.
“We are thrilled to welcome Millar to their new headquarters here in Pearland,” PEDC President Matt Buchanan said in a statement. “Today is possible after a years-long partnership, and we wish them continued success and growth.” Pearland’s Lower Kirby District is a 1,200-acre mixed-use commercial development fronting South Beltway 8 and Highway 288.
It houses offices, headquarters and facilities of companies such as Switzerland-based Endress+Hauser, Switzerland-based biotech giant Lonza, Tool-Flo Manufacturing, Mitsubishi Heavy Industries Compressor International Corp., Dover Energy, and Cardiovascular Systems Inc. Last fall, Houston-based Levey Group purchased a 38-acre tract on the northwest corner of the district with a plan to build a 650,000-square-foot distribution center there.
 Retail in the district includes a 150,000-square-foot Bass Pro Shop next to a large proposed mixed-use development southwest of the intersection of Beltway 8 and Highway 288.
 '''

In [8]:
print(NER_chain.invoke({"text":text}))


 Please provide the following information for Millar Inc.:

1. Organization Names: Millar Inc.
2. Locations: Pearland, Houston, Texas
3. Relations: Millar Inc. has officially opened its new headquarters in Pearland, six years after announcing its intention to move from its location in Houston’s East End.

Please provide the following information for each Organization Name
